In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import yfinance as yf
import datetime
from IPython.display import clear_output
import pytz
import timeit
#import plotly.io as pio
#pio.renderers.default = "browser"
# start_date = "2021-12-30"
# end_date = "2021-12-30"
# #intraday_data = yf.download(tickers='SE', start=start_date, end=end_date, interval='1m', auto_adjust=True)
# intraday_data = yf.download(tickers='MSFT', period='2d', interval='1m', auto_adjust=True) 
# #print(intraday_data.loc['2021-12-30 15:59:00-05:00'].head())
# #intraday_data = intraday_data.dropna()
# print(intraday_data)
# print(intraday_data.shape)
# # Define the resampling logic
# ohlcv_dict = {
#      'Open': 'first',
#      'High': 'max',
#      'Low': 'min',
#      'Close': 'last',
#      'Volume': 'sum'
# }

# # 10T for 10 minutes, D for 1 day and M for 1 month

# # Resample the data
# intraday_data = intraday_data.resample('3T').agg(ohlcv_dict)
# print(intraday_data.shape)
# intraday_data = intraday_data.dropna()
# intraday_data.head()

In [2]:
def download_ticker():
    

def resample_data(df, interval):
    # Define the resampling logic
    ohlcv_dict = {
         'Open': 'first',
         'High': 'max',
         'Low': 'min',
         'Close': 'last',
         'Volume': 'sum'
    }
    # 10T for 10 minutes, D for 1 day and M for 1 month
    # Resample the data
    df = df.resample(interval).agg(ohlcv_dict)
    df = df.dropna()
    return df

In [3]:
from pyti.exponential_moving_average import exponential_moving_average as ema
import pandas_ta as ta

# Calculate indicators EMA9, EMA13, VWAP
def vwap(df):
    v = df.Volume.values
    tp = (df.High+df.Low+df.Close).div(3).values
    df['vwap'] = (tp*v).cumsum() / v.cumsum()
    return df

def calculate_indicators(df):
    df = df.groupby(df.index.date, group_keys=False).apply(vwap)
    df['ema_nine'] = ta.ema(df['Close'], length=9)
    df['ema_thirteen'] = ta.ema(df['Close'], length=13)
    #df['ema_nine'] = ema(df['Close'], 9)
    #df['ema_thirteen'] = ema(df['Close'], 13)
    return df
    
#intraday_data = calculate_indicators(intraday_data)
#intraday_data.to_csv('test.csv')
#intraday_data.head(10)





In [4]:
import plotly.graph_objs as go

def create_graph():
    f = go.FigureWidget()
    f.add_candlestick()
    f.add_scatter()
    f.add_scatter()
    f.add_scatter()
    f.update_xaxes(
            rangeslider_visible=True,
            rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2020-12-25", "2021-01-01"])  # hide holidays (Christmas and New Year's, etc)
            ]
        )
    display(f)
    return f
def update_graph(df, f):
    # Update candlestick
    candlestick = f.data[0]
    candlestick.x = x=df.index
    candlestick.open = df['Open']
    candlestick.high = df['High']
    candlestick.low = df['Low']
    candlestick.close = df['Close']
    candlestick.name = 'market data'
    
    #Update VWAP
    scatter_vwap = f.data[1]
    scatter_vwap.x = df.index
    scatter_vwap.y = df['vwap']
    scatter_vwap.mode='lines'
    scatter_vwap.name = 'VWAP'
    scatter_vwap.line = dict(color='teal')
    
    #Update EMA9
    scatter_ema9 = f.data[2]
    scatter_ema9.x = df.index
    scatter_ema9.y = df['ema_nine']
    scatter_ema9.mode='lines'
    scatter_ema9.name = 'EMA9'
    scatter_ema9.line = dict(color='blue')
    
    #Update EMA13
    scatter_ema13 = f.data[3]
    scatter_ema13.x = df.index
    scatter_ema13.y = df['ema_thirteen']
    scatter_ema13.mode='lines'
    scatter_ema13.name = 'EMA13'
    scatter_ema13.line = dict(color='purple')
    
    
    return f
def plot_graph(df):
    #clear_output()
    fig = go.Figure()

    fig.add_trace(go.Candlestick(x=df.index,
                                open=df['Open'],
                                high=df['High'],
                                low=df['Low'],
                                close=df['Close'], name = 'market data'))
    fig.add_trace(go.Scatter(x=df.index, y=df['vwap'], mode='lines', name = 'VWAP', 
                  line=dict(color='teal')))
    fig.add_trace(go.Scatter(x=df.index, y=df['ema_nine'], mode='lines', name = 'EMA9', 
                  line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df.index, y=df['ema_thirteen'], mode='lines', name = 'EMA13', 
                  line=dict(color='purple')))

    #fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_xaxes(
            rangeslider_visible=True,
            rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2020-12-25", "2021-01-01"])  # hide holidays (Christmas and New Year's, etc)
            ]
        )
    display(fig)

In [5]:
# EMA9 + VWAP Combo
# def generate_ema9_vwap_signal(df):
#     # Looking for Bearish Crossover
#     df['bearish_position'] = df['vwap'] > df['ema_nine']
#     df['bearish_pre_position'] = df['bearish_position'].shift(1)
#     #df.dropna(inplace=True) # dropping the NaN values
#     df['bearish_crossover'] = np.where((df['bearish_position'] - df['bearish_pre_position'])==1, -1, 0)
#     # Looking for Bullish Crossover
#     df['bullish_position'] = df['ema_nine'] < df['vwap']
#     df['bullish_pre_position'] = df['bullish_position'].shift(1)
#     #df.dropna(inplace=True) # dropping the NaN values
#     df['bullish_crossover'] = np.where((df['bullish_position'] - df['bullish_pre_position'])==-1, 1, 0)
#     df['crossover'] = df["bearish_crossover"] + df["bullish_crossover"]
#     return df

def generate_ema9_vwap_signal(df):
    # Looking for EMA9 and VWAP Crossover
    #df_temp = df.copy()
    #df_temp['crossover_signal'] = 0
    df['crossover_signal'] = np.where(df['ema_nine'] > df['vwap'], 1, 0)
    df['crossover_position'] = df['crossover_signal'].diff()    
    return df

def hold_above_or_below(df):
    
    close_to_ema_tolerance = 0.0005 # 0.05%
    
    # Check if last 5 candles are within tolerance    
    df['triple_ema_low_closeness'] = abs(df['Low'] - df['ema_nine']) / df['ema_nine']
    df['triple_ema_hold_above_signal'] = np.where((df['Low'] >= df['ema_nine']) | (df['triple_ema_low_closeness'] <= close_to_ema_tolerance) , 1, 0)
    #df['triple_ema_hold_above_signal'] = np.where(df['Low'] >= df['ema_nine'], 1, 0)
    df['triple_ema_hold_above_signal_cummulative'] = df['triple_ema_hold_above_signal'].cumsum()
    if (df.loc[df.index[-1], 'triple_ema_hold_above_signal_cummulative'] == 5):
        df.loc[df.index[-1], 'triple_ema_hold_rule_one'] = 1
        
        
    # Check if rule 1 fulfilled - 5 Candles below EMA 20
    df['triple_ema_high_closeness'] = abs(df['High'] - df['ema_nine']) / df['ema_nine']
    df['triple_ema_hold_below_signal'] = np.where((df['High'] <= df['ema_nine']) | (df['triple_ema_high_closeness'] <= close_to_ema_tolerance), -1, 0)
    #df['triple_ema_hold_below_signal'] = np.where(df['High'] <= df['ema_nine'], -1, 0)

    df['triple_ema_hold_below_signal_cummulative'] = df['triple_ema_hold_below_signal'].cumsum()
    if (df.loc[df.index[-1], 'triple_ema_hold_below_signal_cummulative'] == -5):
        df.loc[df.index[-1], 'triple_ema_hold_rule_one'] = -1
    
    tolerance  = 0.0005 # 0.05%
    # Check for Long or Short
    if df.loc[df.index[-1], 'triple_ema_hold_rule_one'] == -1:
    
        # Check if rule 2 fulfilled - candle 0 touches/nears EMA20, 0.05% is considered near
        if 0 <= (abs(df.loc[df.index[-1], 'ema_nine'] - df.loc[df.index[-1], 'High'])
                 /df.loc[df.index[-1], 'ema_nine']) <= tolerance:
            df.loc[df.index[-1], 'triple_ema_retest_rule_two'] = -1 
                
    
    elif df.loc[df.index[-1], 'triple_ema_hold_rule_one'] == 1:
        
        # Check if rule 2 fulfilled - candle 0 touches/nears EMA20, 0.05% is considered near
        if 0 <= (abs(df.loc[df.index[-1], 'ema_nine'] - df.loc[df.index[-1], 'Low'])
                 /df.loc[df.index[-1], 'ema_nine']) <= tolerance:
            df.loc[df.index[-1], 'triple_ema_retest_rule_two'] = 1
    
    return df
    
def triple_ema(df):
    #df_temp = df.copy()
    #df_temp['hold_rule_one'] = 0
    #df_temp['retest_rule_two'] = 0
    
    if(df.shape[0] >= 5):
        print("Check for TripleEMA Setup")
        # Check if last 5 candles are above EMA9
        df_last_5 = df.iloc[-5:]
        df_last_5 = hold_above_or_below(df_last_5)
        # Update last row in main df
        df.iloc[-1] = df_last_5.iloc[-1]

    return df
    
    
def vwap_retest(df):
    
    # Check for vwap retest
    tolerance  = 0.002 # 0.05%
    candlewick_midpoint = (df.loc[df.index[-1], 'Open'] + df.loc[df.index[-1], 'Close'])/2
    df.loc[df.index[-1], 'candle_open_close_midpoint'] = candlewick_midpoint
    #candle_midpoint = abs(df.loc[df.index[-1], 'Close'] + df.loc[df.index[-1], 'Open'])/2
    
    # Update Local Max and Min value
    if df.loc[df.index[-1], 'High'] > df.loc[df.index[-1], 'current_vwap_local_max']:
        df.loc[df.index[-1], 'current_vwap_local_max'] = df.loc[df.index[-1], 'High']
        
    if df.loc[df.index[-1], 'Low'] < df.loc[df.index[-1], 'current_vwap_local_min']:
        df.loc[df.index[-1], 'current_vwap_local_min'] = df.loc[df.index[-1], 'Low']
        
    if df.shape[0] > 4 :
        # Determine the last 5 candles, whether they are above or below vwap
        df_last_5 = df.iloc[-5:]
        no_last_5_candle_pos_above = df_last_5[df_last_5['candle_open_close_midpoint'] >= df_last_5['vwap']].shape[0]
        no_last_5_candle_pos_below = df_last_5[df_last_5['candle_open_close_midpoint'] <= df_last_5['vwap']].shape[0]
        
        # Determine the last two candles' price action
        price_action_direction  = df.loc[df.index[-1], 'Close'] - df.loc[df.index[-2], 'Close']
        
        # When candle is above vwap and price action is still decreasing, it's in the direction to retest vwap
        if no_last_5_candle_pos_above >= 3 and  price_action_direction < 0:
            print("Enter LONG VWAP Retest")
            # Check if the candle's close / low is near vwap
            if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Low'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance):
                    df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = 1
                    
        # When candle is below vwap and price action is still increasing, it's in the direction to retest vwap
        #candlewick_midpoint <= df.loc[df.index[-1], 'vwap']
        elif no_last_5_candle_pos_below >= 3 and  price_action_direction > 0:
            # Check if candle's close / high is near vwap
            if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'High'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance):
                    df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = -1
        
        # Only if vwap retest rule 1 is fulfilled, check for confirmation
        if df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] != 0:
            # For Long setup, check if latest candle's close is above the previous candle's close and it closes above vwap
            if (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 1) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-2], 'Close']) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-1], 'vwap']):
                    df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
            # For Short setup, check if confilatestrmation candle's close is below the previous candle's close and it closes below vwap
            elif (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == -1) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-2], 'Close']) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-1], 'vwap']):
                    df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
            
#         # Check if previous candle has fulfilled vwap_test
#         if df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 0:
#             # Check if latest candle is above or below vwap (using midpoint between HIGH and LOW to decide)
#             # and the price action is on a downtrend
#             # Candle above vwap (Long)
#             if candlewick_midpoint > df.loc[df.index[-1], 'vwap'] :
#                 # Check if candle's low is near vwap
#                 #if (0 <= (abs(df.loc[df.index[-1], 'Low'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance)  or (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Open'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 and price_action_direction < 0:
#                     df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = 1

#             # Candle below vwap (Short) and price action is on a uptrend
#             elif candlewick_midpoint < df.loc[df.index[-1], 'vwap'] :
#                 # Check if candle's high is near vwap
#                 #if (0 <= (abs(df.loc[df.index[-1], 'High'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Open'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 and price_action_direction > 0:
#                     df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = -1

#         else:
#             # For Long setup, check if latest candle's close is above the previous candle's close
#             if (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 1) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-2], 'Close']):
#                     df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
#             # For Short setup, check if confilatestrmation candle's close is below the previous candle's close
#             elif (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == -1) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-2], 'Close']):
#                     df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
    
    # Append to result dataframe
    #result_df = result_df.append(df.iloc[-1])
    return df    

In [21]:
# Retrieve ticker's symbol from watchlist
watchlist_filename = "./Watchlist_test.csv"
df_watchlist = pd.read_csv(watchlist_filename)  
print(df_watchlist)

# Create a dictionary 
watchlist_dict = pd.Series(None,df_watchlist.Symbol.values).to_dict()
# Initialize empty dataframe
for key, df in watchlist_dict.items():
    watchlist_dict[key] = pd.DataFrame()



    
# Simulate Live Data
intraday_data = yf.download(tickers='MSFT', period='4d', interval='1m', auto_adjust=True)
#intraday_data.to_csv('SE_14012022_VWAP_RETEST_SHORT.csv')
#intraday_data.index= intraday_data.index.tz_localize(None)
intraday_data = resample_data(intraday_data, '3T')
intraday_data = calculate_indicators(intraday_data)

today_date = datetime.date(2022, 1, 19) # to be replaced by datetime.now()
today_date_str = today_date.strftime("%Y-%m-%d")
print(intraday_data.shape)
# Narrow to today's
intraday_df_today = intraday_data.loc[today_date_str]

index = 0
interval = 3
max_length = 130 # 3mins chart max length per day for 3mins chart
f = create_graph()
result_df = pd.DataFrame()
time_list = []
triple_ema_time_list = []
ema9_crossover_time_list = []
vwap_time_list = []
while (index < max_length):
    start_time = timeit.default_timer()
    
    # Calculate the indicators
    #intraday_data = calculate_indicators(intraday_data)
    # Narrow to today's
    #intraday_df = intraday_data.loc[today_date_str]
    #print(intraday_df)
    # Narrow to current index
    datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval) 
    #eastern_tzinfo = pytz.timezone("US/Eastern")
    #datetime_time = eastern_tzinfo.localize(datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval), is_dst=None)
    # Convert datetime to string
    #datetime_time_str = datetime_time.strftime('%Y-%m-%d %H:%M:%S%z')
    print(datetime_time)                                        
    intraday_df = intraday_df_today.loc[:datetime_time]
    
    # Create columns for Triple EMA
    intraday_df['triple_ema_low_closeness'] = 0
    intraday_df['triple_ema_high_closeness'] = 0
    intraday_df['triple_ema_hold_rule_one'] = 0
    intraday_df['triple_ema_retest_rule_two'] = 0
    intraday_df['triple_ema_hold_above_signal'] = 0
    intraday_df['triple_ema_hold_above_signal_cummulative'] = 0
    intraday_df['triple_ema_hold_below_signal'] = 0
    intraday_df['triple_ema_hold_below_signal_cummulative'] = 0 
    
    # Create columns for EMA 9 and VWAP crossover
    intraday_df['crossover_signal'] = 0
    intraday_df['crossover_position'] = 0
      
    # Create columns for vwap_retest
    intraday_df['vwap_retest_signal_rule_one'] = 0
    intraday_df['vwap_confirmation_candle_signal_rule_two'] = False
    intraday_df['candle_open_close_midpoint'] = 0
    intraday_df['current_vwap_local_max'] = 0
    intraday_df['prev_vwap_local_max'] = 0
    intraday_df['current_vwap_local_min'] = np.inf
    intraday_df['prev_vwap_local_min'] = np.inf
    
        
    # Append latest row to result_df for processing
    result_df = result_df.append(intraday_df.iloc[-1])
    result_df['vwap_confirmation_candle_signal_rule_two'] = result_df['vwap_confirmation_candle_signal_rule_two'].astype('bool')
    
    # Triple EMA
    temp_start_time = timeit.default_timer()
    result_df = triple_ema(result_df)
    triple_ema_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())
    
    # EMA9 and VWAP crossover
    temp_start_time = timeit.default_timer()
    result_df = generate_ema9_vwap_signal(result_df)
    ema9_crossover_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())
    
    # Test for VWAP retest
    temp_start_time = timeit.default_timer()
    result_df = vwap_retest(result_df)
    vwap_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())
    #print(result_df.tail(5))
    #display(result_df.loc[result_df['vwap_retest_signal_rule_one'] == 1].head(10))
    #display(result_df.loc[result_df['vwap_confirmation_candle_signal_rule_two'] == True].head(10))
    
    time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - start_time)).total_seconds())
    # Plot Graph
    f = update_graph(intraday_df, f)
    #print(index)
    index+=1
    


    Exchange Symbol
0     NASDAQ   AAPL
1     NASDAQ   ABNB
2     NASDAQ   ADBE
3     NASDAQ   AMZN
4       NYSE   BABA
..       ...    ...
134     NYSE    YUM
135   NASDAQ     ZM
136     NYSE    PLD
137     NYSE    UNH
138     NYSE   ALLY

[139 rows x 2 columns]


NameError: name 'yf' is not defined

In [7]:
result_df.loc[result_df['crossover_position'] == 1]

,Close,High,Low,Open,Volume,candle_open_close_midpoint,crossover_position,crossover_signal,current_vwap_local_max,current_vwap_local_min,...,triple_ema_hold_above_signal,triple_ema_hold_above_signal_cummulative,triple_ema_hold_below_signal,triple_ema_hold_below_signal_cummulative,triple_ema_hold_rule_one,triple_ema_low_closeness,triple_ema_retest_rule_two,vwap,vwap_confirmation_candle_signal_rule_two,vwap_retest_signal_rule_one
2022-01-19 09:54:00-05:00,313.149994,313.350006,312.500000,312.500000,633681.0,312.824997,1.0,1,313.350006,312.500000,...,1.0,5.0,0.0,0.0,1.0,0.007625,0.0,309.737881,False,0.0
2022-01-19 12:24:00-05:00,310.804993,310.825012,309.549988,309.559998,296559.0,310.182495,1.0,1,310.825012,309.549988,...,0.0,4.0,0.0,0.0,0.0,0.000521,0.0,309.656730,True,0.0


In [8]:
result_df.loc[result_df['crossover_position'] == -1]

,Close,High,Low,Open,Volume,candle_open_close_midpoint,crossover_position,crossover_signal,current_vwap_local_max,current_vwap_local_min,...,triple_ema_hold_above_signal,triple_ema_hold_above_signal_cummulative,triple_ema_hold_below_signal,triple_ema_hold_below_signal_cummulative,triple_ema_hold_rule_one,triple_ema_low_closeness,triple_ema_retest_rule_two,vwap,vwap_confirmation_candle_signal_rule_two,vwap_retest_signal_rule_one
2022-01-19 10:36:00-05:00,309.420013,310.285004,309.350006,310.190002,476815.0,309.805008,-1.0,0,310.285004,309.350006,...,0.0,0.0,-1.0,-5.0,-1.0,0.003684,0.0,310.673172,True,0.0
2022-01-19 13:48:00-05:00,309.450989,309.619995,309.059998,309.589996,146524.0,309.520493,-1.0,0,309.619995,309.059998,...,0.0,0.0,-1.0,-5.0,-1.0,0.002284,-1.0,309.809246,True,0.0


In [9]:
result_df.loc[result_df['triple_ema_hold_rule_one'] == -1]

,Close,High,Low,Open,Volume,candle_open_close_midpoint,crossover_position,crossover_signal,current_vwap_local_max,current_vwap_local_min,...,triple_ema_hold_above_signal,triple_ema_hold_above_signal_cummulative,triple_ema_hold_below_signal,triple_ema_hold_below_signal_cummulative,triple_ema_hold_rule_one,triple_ema_low_closeness,triple_ema_retest_rule_two,vwap,vwap_confirmation_candle_signal_rule_two,vwap_retest_signal_rule_one
2022-01-19 10:33:00-05:00,310.219604,310.589996,309.859985,309.940002,300341.0,310.079803,0.0,1,310.589996,309.859985,...,0.0,0.0,-1.0,-5.0,-1.0,0.002904,0.0,310.707304,False,-1.0
2022-01-19 10:36:00-05:00,309.420013,310.285004,309.350006,310.190002,476815.0,309.805008,-1.0,0,310.285004,309.350006,...,0.0,0.0,-1.0,-5.0,-1.0,0.003684,0.0,310.673172,True,0.0
2022-01-19 10:39:00-05:00,308.950012,309.579987,308.882812,309.459991,350111.0,309.205002,0.0,0,309.579987,308.882812,...,0.0,0.0,-1.0,-5.0,-1.0,0.004199,0.0,310.636429,False,0.0
2022-01-19 10:42:00-05:00,309.338593,309.359985,308.540009,308.940002,412898.0,309.139297,0.0,0,309.359985,308.540009,...,0.0,0.0,-1.0,-5.0,-1.0,0.004761,0.0,310.593699,False,0.0
2022-01-19 10:45:00-05:00,308.229889,309.515015,308.220001,309.269989,334945.0,308.749939,0.0,0,309.515015,308.220001,...,0.0,0.0,-1.0,-5.0,-1.0,0.004646,-1.0,310.551476,False,0.0
2022-01-19 10:48:00-05:00,308.500000,308.523010,307.820007,308.200012,401123.0,308.350006,0.0,0,308.523010,307.820007,...,0.0,0.0,-1.0,-5.0,-1.0,0.005193,0.0,310.493763,False,0.0
2022-01-19 10:51:00-05:00,308.434998,308.769989,308.350006,308.500000,358100.0,308.467499,0.0,0,308.769989,308.350006,...,0.0,0.0,-1.0,-5.0,-1.0,0.002841,0.0,310.449930,False,0.0
2022-01-19 10:54:00-05:00,308.329987,308.850006,308.070007,308.429413,292732.0,308.379700,0.0,0,308.850006,308.070007,...,0.0,0.0,-1.0,-5.0,-1.0,0.003167,0.0,310.413707,False,0.0
2022-01-19 10:57:00-05:00,307.915009,308.389893,307.320007,308.369995,367336.0,308.142502,0.0,0,308.389893,307.320007,...,0.0,0.0,-1.0,-5.0,-1.0,0.004864,0.0,310.358192,False,0.0
2022-01-19 11:00:00-05:00,308.029999,308.494995,307.720001,307.899994,358315.0,307.964996,0.0,0,308.494995,307.720001,...,0.0,0.0,-1.0,-5.0,-1.0,0.003057,0.0,310.310648,False,0.0


In [10]:
result_df.loc[result_df['triple_ema_retest_rule_two'] == -1]

,Close,High,Low,Open,Volume,candle_open_close_midpoint,crossover_position,crossover_signal,current_vwap_local_max,current_vwap_local_min,...,triple_ema_hold_above_signal,triple_ema_hold_above_signal_cummulative,triple_ema_hold_below_signal,triple_ema_hold_below_signal_cummulative,triple_ema_hold_rule_one,triple_ema_low_closeness,triple_ema_retest_rule_two,vwap,vwap_confirmation_candle_signal_rule_two,vwap_retest_signal_rule_one
2022-01-19 10:45:00-05:00,308.229889,309.515015,308.220001,309.269989,334945.0,308.749939,0.0,0,309.515015,308.220001,...,0.0,0.0,-1.0,-5.0,-1.0,0.004646,-1.0,310.551476,False,0.0
2022-01-19 11:03:00-05:00,308.079987,308.700012,307.820007,308.019989,292260.0,308.049988,0.0,0,308.700012,307.820007,...,0.0,0.0,-1.0,-5.0,-1.0,0.002356,-1.0,310.275297,False,0.0
2022-01-19 11:06:00-05:00,307.820007,308.260010,307.720001,308.040009,186583.0,307.930008,0.0,0,308.260010,307.720001,...,0.0,0.0,-1.0,-5.0,-1.0,0.002210,-1.0,310.250519,False,0.0
2022-01-19 13:42:00-05:00,309.529999,309.760010,309.347992,309.489990,155275.0,309.509995,0.0,1,309.760010,309.347992,...,0.0,0.0,-1.0,-5.0,-1.0,0.001825,-1.0,309.812215,False,-1.0
2022-01-19 13:45:00-05:00,309.579987,309.760010,309.380005,309.540009,106843.0,309.559998,0.0,1,309.760010,309.380005,...,0.0,0.0,-1.0,-5.0,-1.0,0.001507,-1.0,309.811362,False,-1.0
2022-01-19 13:48:00-05:00,309.450989,309.619995,309.059998,309.589996,146524.0,309.520493,-1.0,0,309.619995,309.059998,...,0.0,0.0,-1.0,-5.0,-1.0,0.002284,-1.0,309.809246,True,0.0
2022-01-19 13:51:00-05:00,309.540009,309.769989,309.320007,309.440002,134899.0,309.490005,0.0,0,309.769989,309.320007,...,0.0,0.0,-1.0,-5.0,-1.0,0.001298,-1.0,309.808059,False,-1.0
2022-01-19 14:33:00-05:00,308.215698,308.290009,307.600006,307.750000,170952.0,307.982849,0.0,0,308.290009,307.600006,...,0.0,0.0,-1.0,-5.0,-1.0,0.002015,-1.0,309.725117,False,0.0


In [11]:
# Pull Live Data
intraday_data_new = yf.download(tickers='TWTR', period='2d', interval='1m', auto_adjust=True) 
intraday_data_new.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2022-01-21 15:55:00-05:00,34.884998,34.889999,34.820000,34.834999,209581
2022-01-21 15:56:00-05:00,34.834999,34.845001,34.794998,34.820000,212382
2022-01-21 15:57:00-05:00,34.814999,34.910000,34.814999,34.904999,150360
2022-01-21 15:58:00-05:00,34.900101,34.939999,34.895000,34.935001,202027
2022-01-21 15:59:00-05:00,34.939999,34.939999,34.799999,34.820000,541257


In [12]:
test_intraday_data = yf.download(tickers='MSFT', period='1d', interval='1m', auto_adjust=True)
test_intraday_data.head()
datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0))
test_intraday_data.loc[:datetime_time].head(10)

[*********************100%***********************]  1 of 1 completed


TypeError: Cannot compare tz-naive and tz-aware datetime-like objects

In [ ]:
datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 51, 0)) 
#eastern_tzinfo = pytz.timezone("US/Eastern")
#datetime_time = eastern_tzinfo.localize(datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval), is_dst=None)
# Convert datetime to string
#datetime_time_str = datetime_time.strftime('%Y-%m-%d %H:%M:%S%z')
print(datetime_time)                                        
result_df_temp = result_df.loc[datetime_time:]
result_df_temp.head()

In [29]:
from statistics import mean
from matplotlib import pyplot  as plt
%matplotlib

mean_time_taken = round(mean(time_list),4)

fig = plt.figure()
plt.plot(time_list,'g', label='All 3 Methods')
plt.plot(triple_ema_time_list,'b', label='Triple EMA')
plt.plot(ema9_crossover_time_list,'k', label='EMA9 + VWAP Combo')
plt.plot(vwap_time_list,'r', label='VWAP Retest')
plt.axhline(y = mean_time_taken, color = 'm', linestyle = '--')
plt.xlabel('Iterations')
plt.ylabel('Time Taken (seconds)')
plt.title('Processing time taken for 1 ticker across each iteration')
plt.grid(True)
plt.legend(["All 3 Methods", "Triple EMA", "EMA9 + VWAP Combo", "VWAP Retest", 'Mean Time Taken'], loc="upper left")
plt.show()

from statistics import mean
round(mean(time_list),4)

Using matplotlib backend: Qt5Agg


0.0305

In [31]:
# Retrieve ticker's symbol from watchlist
watchlist_filename = "./Watchlist.csv"
df_watchlist = pd.read_csv(watchlist_filename)  
print(df_watchlist)

# Create a dictionary 
watchlist_dict = pd.Series(None,df_watchlist.Symbol.values).to_dict()
print(watchlist_dict)
# Initialize empty dataframe
for key, df in watchlist_dict.items():
    watchlist_dict[key] = pd.DataFrame()
print(watchlist_dict)

    Exchange Symbol
0     NASDAQ   AAPL
1     NASDAQ   ABNB
2     NASDAQ   ADBE
3     NASDAQ   AMZN
4       NYSE   BABA
..       ...    ...
134     NYSE    YUM
135   NASDAQ     ZM
136     NYSE    PLD
137     NYSE    UNH
138     NYSE   ALLY

[139 rows x 2 columns]
{'AAPL': nan, 'ABNB': nan, 'ADBE': nan, 'AMZN': nan, 'BABA': nan, 'BIDU': nan, 'COIN': nan, 'CRM': nan, 'CRWD': nan, 'DDOG': nan, 'FB': nan, 'JD': nan, 'MARA': nan, 'MSFT': nan, 'MSTR': nan, 'NFLX': nan, 'PLTR': nan, 'PSTG': nan, 'RBLX': nan, 'ROKU': nan, 'SE': nan, 'SNAP': nan, 'SPOT': nan, 'SQ': nan, 'TWTR': nan, 'U': nan, 'UBER': nan, 'UPST': nan, 'AMAT': nan, 'AMD': nan, 'AVGO': nan, 'INTC': nan, 'MU': nan, 'NVDA': nan, 'QCOM': nan, 'TSM': nan, 'ARKG': nan, 'BNGO': nan, 'BNTX': nan, 'CLOV': nan, 'JNJ': nan, 'MRNA': nan, 'OBSV': nan, 'PFE': nan, 'SENS': nan, 'TDOC': nan, 'TYME': nan, 'BLNK': nan, 'CHPT': nan, 'F': nan, 'LCID': nan, 'LI': nan, 'NIO': nan, 'PLUG': nan, 'RIVN': nan, 'TSLA': nan, 'WKHS': nan, 'XPEV': nan, 'CHWY

<ipython-input-31-2eb6dbd0331f>:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  watchlist_dict = pd.Series(None,df_watchlist.Symbol.values).to_dict()


{'Exchange': 0    NASDAQ
Name: Exchange, dtype: object, 'Symbol': 0    MSFT
Name: Symbol, dtype: object}


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(
